In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
import scipy.stats as stats
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
D = pd.read_csv('train.csv').drop(columns=['id'])
D.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,10320.000000,10320.000000,10320.000000,10320.000000,10210.000000,10320.000000,10320.000000,10320.000000,10320.000000
mean,-119.593791,35.647997,28.544477,2652.874031,541.297845,1431.618120,502.993605,3.882051,208222.779845
std,2.005209,2.132319,12.605356,2215.969393,427.866565,1144.035957,388.016806,1.902330,116152.666840
min,-124.350000,32.540000,1.000000,6.000000,2.000000,8.000000,2.000000,0.499900,14999.000000
25%,-121.810000,33.940000,18.000000,1461.000000,294.250000,786.000000,278.000000,2.583300,120000.000000
50%,-118.535000,34.270000,29.000000,2138.000000,438.000000,1165.000000,413.000000,3.550000,180700.000000
75%,-118.010000,37.720000,37.000000,3144.250000,648.750000,1724.000000,607.000000,4.733575,267225.000000
max,-114.550000,41.920000,52.000000,39320.000000,6445.000000,28566.000000,6082.000000,15.000100,500001.000000


In [ ]:
D.isnull().any()

longitude             False
latitude              False
housing_median_age    False
total_rooms           False
total_bedrooms         True
population            False
households            False
median_income         False
ocean_proximity       False
median_house_value    False
dtype: bool

In [ ]:
D.fillna(D['total_bedrooms'].mean(),inplace=True)
#replace nulls with the mean

In [ ]:
#handleing catigorical feautures for training

df=pd.concat([pd.get_dummies(D['ocean_proximity'],drop_first=False),D],axis=1).drop('ocean_proximity',axis=1)
df

,<1h_ocean,inland,island,near_bay,near_ocean,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,0,0,0,1,0,-122.22,37.77,40.0,494.0,114.0,547.0,135.0,2.8015,114800.0
1,0,0,0,1,0,-122.29,37.76,18.0,2873.0,763.0,1243.0,663.0,5.1702,265400.0
2,1,0,0,0,0,-118.48,34.23,29.0,3354.0,707.0,1752.0,650.0,4.5484,239900.0
3,0,0,0,1,0,-122.19,37.80,48.0,1694.0,259.0,610.0,238.0,4.7440,257300.0
4,0,0,0,0,1,-121.93,36.63,33.0,1740.0,342.0,638.0,329.0,3.1912,319800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10315,1,0,0,0,0,-117.79,33.70,16.0,1416.0,249.0,636.0,244.0,5.1741,227700.0
10316,0,1,0,0,0,-117.41,33.95,37.0,1586.0,283.0,675.0,305.0,2.9583,132100.0
10317,1,0,0,0,0,-118.65,34.20,23.0,7480.0,1084.0,3037.0,1058.0,6.9223,338400.0
10318,0,1,0,0,0,-118.00,34.12,37.0,1340.0,325.0,928.0,333.0,3.9219,175000.0


In [ ]:
#outliers were kept after experimentation showing increased accuracy 
z = np.abs(stats.zscore(df))
df = df[(z < 3).all(axis=1)]

In [ ]:
'''
data processing reduced accuracy greatly in the cross validation score. polynomial transformation reduced accuracy the most
despite hacing the best cross validation score.

'''

target = df.columns[12]
features = df.columns[:].drop(target)
X = df.loc[:, features]
Y = df.loc[:, target]
# Standardaizing the data
#X = preprocessing.Normalizer().fit(X).transform(X)
#X = preprocessing.StandardScaler().fit(X).transform(X)
#X = preprocessing.MinMaxScaler().fit(X).transform(X)
#X = preprocessing.PolynomialFeatures(degree=2).fit(X).transform(X)
#X = pd.DataFrame(preprocessing.QuantileTransformer(output_distribution='normal').fit_transform(X))
target

'median_house_value'

In [ ]:
X.shape

(10320, 12)

In [ ]:
from sklearn.metrics import mean_squared_error
import math
from sklearn.model_selection import cross_val_score


dataSplits = []
max = 10000000000000
index = 0
scores = []
for i in range(100):
    dataStuff = train_test_split(X, Y, test_size=0.2)
    dataSplits.append(dataStuff)
    X_train, X_vlad, Y_train, Y_vlad = dataStuff

    reg = LinearRegression()
    reg.fit(X_train,Y_train)
    predict = reg.predict(X_vlad)
    
    sc = mean_squared_error(Y_vlad,predict, squared=False)
    scores.append(sc)
    if(max > sc):
      max = sc
      index = i
print(max)
print(np.array(scores).mean())

print("---------------------------------------")
S = cross_val_score(reg, X, Y, cv=10, scoring='neg_mean_squared_error')


#print(math.sqrt(abs(S)).mean())
S = np.array(S)
S = np.absolute(S)
S = np.sqrt(S)
print(S.mean())

62589.28794986505
65928.10325552995
---------------------------------------
66052.20937352919


In [ ]:
X_train, X_test, Y_train, Y_vlad = dataSplits[index]
LR = LinearRegression()
LR.fit(X_train,Y_train)
predict = LR.predict(X_test)
sc = mean_squared_error(Y_vlad,predict, squared=False)
print(sc)

62589.28794986505


In [ ]:
uploaded = files.upload()

Saving test.csv to test (1).csv


In [ ]:
T1 = pd.read_csv('test.csv').drop(columns=['id'])



In [ ]:
T1.fillna(D['total_bedrooms'].mean(),inplace=True)
T1['ocean_proximity'] = T1['ocean_proximity'].str.lower()
T1['ocean_proximity'] = T1['ocean_proximity'].str.replace(" ","_")
df=pd.concat([pd.get_dummies(T1['ocean_proximity'],drop_first=False),T1],axis=1).drop('ocean_proximity',axis=1)
df.drop(columns=['island'], inplace=True)
'''
df['income per working population']=df['median_income']/(df['population']-df['households'])
df['bed per house']=df['total_bedrooms']/df['total_rooms']
df['h/p']=df['households']/df['population']

def type_building(x):
    if x<=10:
        return "new"
    elif x<=30:
        return 'mid old'
    else:
        return 'old'
df=pd.concat([df,pd.get_dummies(df['housing_median_age'].apply(type_building),drop_first=False)],axis=1)
'''
#D.drop(columns=['total_bedrooms','ocean_proximity'], inplace=True)
#D
df

,<1h_ocean,inland,near_bay,near_ocean,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,0,0,1,0,-122.36,37.94,41.0,2591.0,585.0,1638.0,462.0,1.8220
1,1,0,0,0,-118.37,34.16,17.0,4150.0,1148.0,1808.0,1041.0,3.5051
2,1,0,0,0,-118.29,33.95,40.0,2808.0,695.0,2357.0,627.0,1.9655
3,0,1,0,0,-121.77,38.01,13.0,2983.0,534.0,1417.0,510.0,3.9861
4,1,0,0,0,-118.26,34.08,45.0,2174.0,627.0,1992.0,557.0,2.5428
...,...,...,...,...,...,...,...,...,...,...,...,...
10315,0,0,1,0,-122.15,37.74,49.0,1325.0,277.0,764.0,282.0,3.3125
10316,1,0,0,0,-118.27,33.94,39.0,2078.0,561.0,1901.0,504.0,1.1468
10317,1,0,0,0,-122.59,38.43,20.0,2791.0,546.0,785.0,512.0,3.4561
10318,1,0,0,0,-122.04,37.37,33.0,2757.0,489.0,1201.0,481.0,5.0453


In [ ]:
#T1.drop(columns=['total_bedrooms','ocean_proximity'], inplace=True)
#T1 = preprocessing.PolynomialFeatures(degree=2).fit(T1).transform(T1)
#T1.fillna(D['total_bedrooms'].median(),inplace=True)
#T1.isnull().any()
df.shape

(10320, 18)

In [ ]:
df.shape

(10320, 18)

In [ ]:
LR = LinearRegression()
LR.fit(X_train,Y_train)
predict = LR.predict(df)

In [ ]:
final = pd.read_csv('test.csv')
final_data = pd.DataFrame({'id':final.id,'median_house_value':predict})
final_data.to_csv('Final_submit.csv',index=False)

In [ ]:
files.download("Final_submit.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>